In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.2 MB/s 
     |████████████████████████████████| 895 kB 35.3 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.5 MB 18.5 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 1.2 MB 24.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=71f2dee3f18f2712db8cd9375a6c5cb15667dc60faa7b12987dc5970772578c2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [3]:
!git clone https://github.com/AdarshKumar712/NLP_Shared_Task_2022

Cloning into 'NLP_Shared_Task_2022'...
remote: Enumerating objects: 45988, done.
remote: Counting objects: 100% (8470/8470), done.
remote: Compressing objects: 100% (8294/8294), done.
remote: Total 45988 (delta 179), reused 8466 (delta 176), pack-reused 37518
Receiving objects: 100% (45988/45988), 640.61 MiB | 13.39 MiB/s, done.
Resolving deltas: 100% (1327/1327), done.
Checking out files: 100% (48295/48295), done.


In [4]:
cd NLP_Shared_Task_2022/

/content/NLP_Shared_Task_2022


In [5]:
import pandas as pd
df = pd.read_csv("eval_with_result.csv")

In [6]:
df.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,GEO,ENT,TIME,NAR,Overall,STYLE,TONE
0,en,en,1484189203_1484121193,https://wsvn.com/news/local/broward/police-2-m...,https://wsvn.com/news/local/no-swim-advisory-l...,https://web.archive.org/web/https://wsvn.com/n...,https://web.archive.org/web/https://wsvn.com/n...,1.5,4.0,2.0,4.0,3.5,1.0,1.500000
1,en,en,1484011097_1484011106,https://www.zdnet.com/article/autoclerk-databa...,https://securityboulevard.com/2019/10/best-wes...,https://web.archive.org/web/https://www.zdnet....,https://web.archive.org/web/https://securitybo...,1.0,2.0,1.0,1.0,1.0,3.5,2.500000
2,en,en,1484039488_1484261803,https://www.presstelegram.com/2019/12/31/ducks...,https://boingboing.net/2020/01/01/woody-guthri...,https://web.archive.org/web/https://www.presst...,https://web.archive.org/web/https://boingboing...,5.0,4.0,3.0,4.0,4.0,4.0,3.666667
3,en,en,1484332324_1484796748,https://www.financialexpress.com/india-news/we...,https://www.news18.com/news/india/not-the-firs...,https://web.archive.org/web/https://www.financ...,https://web.archive.org/web/https://www.news18...,1.0,1.5,1.0,1.5,1.5,1.5,2.000000
4,en,en,1484012256_1484419682,https://www.birminghammail.co.uk/whats-on/musi...,http://m.fightbacknews.org/2020/1/1/top-20-fil...,https://web.archive.org/web/https://www.birmin...,https://web.archive.org/web/http://m.fightback...,4.0,4.0,1.0,2.5,4.0,2.5,2.500000


In [7]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from scipy.stats import pearsonr
import json
import torch

In [8]:
device = "cuda"

In [9]:
# Type = "T" -> Normal Transformers and Type = "ST" -> Sentence Transformers
def evaluate(model_name="bert-base-multilingual-cased", type="T"):
  def scale(value):
      value=4 + 1.5*(value-1)
      return value
  print("Model Name: " )
  print("-"*70)
  
  if type!="ST":
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    model.to(device)

    X=[]
    Y=[]
    for i in tqdm(range(len(df))):
        f1, f2 = df["pair_id"][i].split("_")
        folder1 = f1[-2:]
        folder2 = f2[-2:]
        try:
          with open("./eval_data/"+folder1+"/"+f1+".json", "r") as f:
            d1 = json.load(f)

          with open("./eval_data/"+folder2+"/"+f2+".json", "r") as f:
            d2 = json.load(f)
        except:
          #print("hehe")
          continue
        text = d1["text"]
        encoded_input = tokenizer(text, return_tensors='pt', truncation=True)
        encoded_input.to("cuda")
        output = model(**encoded_input)
        o1 = output["pooler_output"].to("cpu")
        text = d2["text"]
        encoded_input = tokenizer(text, return_tensors='pt', truncation=True)
        encoded_input.to("cuda")
        output2 = model(**encoded_input)
        o2 = output2["pooler_output"].to("cpu")
        
        cosine_similarity = torch.matmul(o1, o2.T)/(torch.sqrt(torch.sum(o1*o1))*torch.sqrt(torch.sum(o2*o2))) 
        value = cosine_similarity[0][0].detach()
        
        scaled_value = scale(value)
        X.append(scaled_value)
        Y.append(df["Overall"][i])
    print("Pearson Correlation: ", pearsonr(X, Y)[0])
    print("="*70)
    print()
  else:

    model = SentenceTransformer(model_name)
    model.to("cuda");

    X=[]
    Y=[]
    for i in tqdm(range(len(df))):
        f1, f2 = df["pair_id"][i].split("_")
        folder1 = f1[-2:]
        folder2 = f2[-2:]
        try:
          with open("./eval_data/"+folder1+"/"+f1+".json", "r") as f:
            d1 = json.load(f)

          with open("./eval_data/"+folder2+"/"+f2+".json", "r") as f:
            d2 = json.load(f)
        except:
          #print("hehe")
          continue
        sentences = [d1["text"], d2["text"]]
        embeddings = model.encode(sentences)
        o1 = torch.tensor(embeddings[0])
        o2 = torch.tensor(embeddings[1])
        
        cosine_similarity = torch.matmul(o1, o2.T)/(torch.sqrt(torch.sum(o1*o1))*torch.sqrt(torch.sum(o2*o2))) 
        value = cosine_similarity
        
        scaled_value = scale(value)
        X.append(scaled_value)
        Y.append(df["Overall"][i])
    print("Pearson Correlation: ", pearsonr(X, Y)[0])
    print("="*70)
    print()
    

In [10]:
evaluate()

Model Name: 
----------------------------------------------------------------------


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/4902 [00:00<?, ?it/s]

Pearson Correlation:  -0.0047246708638204885



In [ ]:
evaluate(model_name="xlm-roberta-base")

Model Name: 
----------------------------------------------------------------------


Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/4902 [00:00<?, ?it/s]

Pearson Correlation:  -0.28853263498078124



In [ ]:
evaluate(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", type="ST")

Model Name: 
----------------------------------------------------------------------


  0%|          | 0/4902 [00:00<?, ?it/s]

Pearson Correlation:  -0.5654393219690194



In [ ]:
evaluate(model_name="sentence-transformers/all-mpnet-base-v1", type="ST")

Model Name: 
----------------------------------------------------------------------


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

  0%|          | 0/4902 [00:00<?, ?it/s]

Pearson Correlation:  -0.39703179168477826



# Some further Experiments on Training data

In [ ]:
import pandas as pd
df = pd.read_csv("train_v2.csv")

In [ ]:
df.describe()

,Geography,Entities,Time,Narrative,Overall,Style,Tone
count,4964.000000,4964.000000,4964.000000,4964.000000,4964.000000,4964.000000,4964.000000
mean,1.930625,2.704471,1.956806,2.854857,2.803369,1.746016,1.722173
std,1.148147,1.157783,1.148235,1.110514,1.133874,0.913308,0.869120
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000
50%,1.000000,3.000000,1.333333,3.000000,3.000000,1.333333,1.333333
75%,3.000000,4.000000,3.000000,4.000000,4.000000,2.000000,2.000000
max,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000


In [ ]:
df.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [ ]:
columns = df.describe().columns

for j in range(len(columns)):
  col1 = "Overall"
  col2 = columns[j]
  corr = df[col1].corr(df[col2])
  print("Correaltion between "+ col1 + " and " +col2+": ", corr)

Correaltion between Overall and Geography:  0.5914521782573293
Correaltion between Overall and Entities:  0.8081445471447216
Correaltion between Overall and Time:  0.42628910594144986
Correaltion between Overall and Narrative:  0.8841530696804962
Correaltion between Overall and Overall:  1.0
Correaltion between Overall and Style:  0.325557613619607
Correaltion between Overall and Tone:  0.4451917481115466


In [ ]:
indices = list(df[df["url1_lang"]=="ar"].index)
indices += list(df[df["url1_lang"]=="fr"].index)

In [ ]:
df1 = df.drop(index=indices)

In [ ]:
len(df1)

4618

In [ ]:
l2 = list(df1.sample(n=500).index)
indices += l2

In [ ]:
df_train = df1.drop(index=l2).reset_index(drop=True)
df_val = df.loc[indices,:]
df_val = df_val.reset_index(drop=True)

In [ ]:
df_train.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-...,https://www.washingtonpost.com/world/the_ameri...,https://web.archive.org/web/www.washingtonpost...,https://web.archive.org/web/www.washingtonpost...,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three...,https://www.thestar.com/news/world/europe/2020...,https://web.archive.org/web/www.stlucianewsonl...,https://web.archive.org/web/www.thestar.com/ne...,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-...,https://www.timesofisrael.com/trump-says-he-do...,https://web.archive.org/web/www.teaparty.org/t...,https://web.archive.org/web/www.timesofisrael....,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-mo...,https://www.channelnewsasia.com/news/asia/indi...,https://web.archive.org/web/news.yahoo.com/ind...,https://web.archive.org/web/www.channelnewsasi...,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [ ]:
df_val.head()

,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,ar,ar,1559207788_1558731176,https://www.alquds.co.uk/أدباء-ومثقفون-مغاربة-...,https://arabic.sputniknews.com/arab_world/2020...,https://web.archive.org/web/www.alquds.co.uk/أ...,https://web.archive.org/web/arabic.sputniknews...,4.0,4.0,1.0,3.0,4.0,1.0,1.0
1,ar,ar,1559280063_1558230540,https://arabic.rt.com/press/1097100-%D9%85%D9%...,https://arabic.sputniknews.com/russia/20200325...,https://web.archive.org/web/arabic.rt.com/pres...,https://web.archive.org/web/arabic.sputniknews...,4.0,3.0,1.0,4.0,4.0,1.0,1.0
2,ar,ar,1559218846_1558750714,https://www.djazairess.com/alseyassi/127892,https://arabic.sputniknews.com/arab_world/2020...,https://web.archive.org/web/www.djazairess.com...,https://web.archive.org/web/arabic.sputniknews...,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,ar,ar,1559204879_1558731178,http://www.alkhaleej.ae/Home/GetPage/f86dd8f8-...,https://arabic.sputniknews.com/world/202003251...,https://web.archive.org/web/www.alkhaleej.ae/H...,https://web.archive.org/web/arabic.sputniknews...,3.0,4.0,1.0,4.0,4.0,1.0,2.0
4,ar,ar,1559218867_1558731176,https://www.djazairess.com/elmassa/179258,https://arabic.sputniknews.com/arab_world/2020...,https://web.archive.org/web/www.djazairess.com...,https://web.archive.org/web/arabic.sputniknews...,4.0,4.0,1.0,4.0,4.0,1.0,1.0


In [ ]:
len(df_train), len(df_val)

(4118, 846)

In [ ]:
df_train.to_csv("train.csv", index=False)

In [ ]:
df_val.to_csv("val.csv", index=False)